# Getting Started
An introduction to the Velour evaluation service.

In [ ]:
from velour.client import Dataset, Model, Client, ClientException
from velour.coretypes import Datum, Annotation, Label, GroundTruth, Prediction
from velour.enums import TaskType

1. Getting Started
    a. Basic usage & workflow of classification task
    b. Labels
2. Annotations
    a. Annotations
    b. Image
        i. geometry
    c. Tabular
    d. Future support
3. Metadata
    a. Adding metadata to Dataset, Model, Datum and Annotation objects.
    b. How this enables MetaTypes for representing customer data.
4. Filtering
    a. Basic filters from intrinsics
    b. Leverage metadata to create application-specific filters
    c. Geospatial filters
5. Classification
6. Object Detection
7. Semantic Segmentation

Connect to the API

In [ ]:
client = Client("http://localhost:8000")

Dataset and Model are client-types in Velour. Their internal values are determined by the backend. Note that they use a `classmethod` for creation and that they take the client object as input.

If the normal initializer is used to create a client-type then creation on the backend is not guaranteed.

In [ ]:
dataset = Dataset.create(client, name="myDataset")
model = Model.create(client, name="myModel")

`velour.coretypes` contain other velour types that are used to construct representations of your annotations and metadata. The two most important types are `Datum` and `Annotation`. These are the building blocks for constructing a representation of your data that can be operated over by Velour.

Let's create a datum and then an annotation for a classification job.

In [ ]:
datum = Datum(uid="myDatum1")

annotation = Annotation(
    task_type=TaskType.CLASSIFICATION,
    labels=[
        Label(key="k1", value="v1"),
        Label(key="k2", value="v2"),
    ]
)

annotation_with_scores = Annotation(
    task_type=TaskType.CLASSIFICATION,
    labels=[
        Label(key="k1", value="v1", score=0.7),
        Label(key="k1", value="v2", score=0.3),
        Label(key="k2", value="v1", score=0.1),
        Label(key="k2", value="v2", score=0.9),
    ]
)

`velour.coretypes` also defines `GroundTruth` and `Prediction`. These are closely related objects with the only difference being the backend target.

In [ ]:
groundtruth = GroundTruth(datum, annotations=[annotation])

In [ ]:
prediction = Prediction(datum, annotations=[annotation_with_scores])

We now have everything required to upload to velour! 

In [ ]:
dataset.add_groundtruth(groundtruth)
model.add_prediction(prediction)

Before we can run an evaluation we need to finalize our dataset and then our predictions over the dataset. This is a requirement of Velour so that repeat evaluations over a Dataset-Model pairing are guaranteed to return the same result.

In [ ]:
dataset.finalize()

`Model` finalization is a little different than `Dataset` as it is locking predictions with respect to the dataset they operated over.

In [ ]:
model.finalize_inferences(dataset)

We now have a dataset and model representation on the backend that we can evaluate!

In [ ]:
evaluation = model.evaluate_classification(dataset)
evaluation.wait_for_completion()

We currently support `classification`, `object-detection`, and `semantic-segmentation` evaluations.

In [ ]:
print(evaluation.results)